# Using Selenium and Requests: _Zara_

In this section we will be using selenium and beautiful soup to scrape all of the women's bags from Zara.
First, we will use selenium to get everylink for the bags, and then use Beautiful Soup to get the correct picture,price, product link, and description.

Some great sources used to help in this: 
- [Selenium Website for Locating Elements](https://selenium-python.readthedocs.io/locating-elements.html)
- [Stackover flow to Save Images](https://stackoverflow.com/questions/18497840/beautifulsoup-how-to-open-images-and-download-them)
- [Get all image src's with specific class](https://stackoverflow.com/questions/8289957/python-2-7-beautiful-soup-img-src-extract)


Our steps are as follows: 
1. [first]()
2. [second]()
3. [third]()
4. [fourth]() 

#### Import your needed Libraries:

In [254]:
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import cv2
import numpy as np 
import pandas as pd
import re
import urllib

#### Create variables for each of the URL

We will do this for each of the pages we will be pulling from on Zara.

In [30]:
z_bags = 'https://www.zara.com/us/en/woman-shoes-bags-l1253.html?v1=1180507'

#### Create our Chrome Driver element and get Links:

Here I am using the Chrome Driver Manager because I was facing some issues using the driver, but this resolved all of them.

We will use selenium to first **filter** the page for the types of bags we need, and then will return a list of the **inner HTML** of the items left.

We will then create a **_dictionary_** of the item's: __name (description), price, and url.__

In [416]:
#create the chrome driver
start_time = time.time()
driver = webdriver.Chrome(ChromeDriverManager().install()) 
driver.get(z_bags)

#now, filter the page by the types of bags we want to see
driver.find_element_by_class_name('products-filter').click()
time.sleep(0.5)
driver.find_element_by_class_name('_view-more-button').click()
time.sleep(0.5)
driver.find_element_by_xpath("//div[@data-value='7000000000000733005']").click()
time.sleep(0.5)
driver.find_element_by_xpath("//div[@data-value='7000000000000733505']").click()
time.sleep(0.5)
driver.find_element_by_xpath("//div[@data-value='7000000000000733504']").click()
time.sleep(0.5)
driver.find_element_by_xpath("//div[@data-value='7000000000000030518']").click()
time.sleep(0.5)
driver.find_element_by_xpath("//div[@data-value='7000000000000733006']").click()
time.sleep(0.5)
# driver.find_element_by_xpath("//div[@data-value='7000000000000001578']").click()
time.sleep(1)
driver.find_element_by_xpath("//div[@data-value='7000000000000030514']").click()
time.sleep(0.5)
driver.find_element_by_xpath("//div[@data-value='7000000000000030517']").click()
time.sleep(0.5)
driver.find_element_by_xpath("//div[@data-value='7000000000000030516']").click()
time.sleep(0.5)
driver.find_element_by_xpath("//div[@data-value='7000000000000733007']").click()
time.sleep(0.5)
driver.find_element_by_xpath("//div[@data-value='7000000000000030515']").click()
time.sleep(0.5)
driver.find_element_by_xpath("//div[@data-value='7000000000000001577']").click()
time.sleep(0.5)
driver.find_element_by_xpath("//div[@data-value='7000000000000733503']").click()
time.sleep(0.5)

#create an html item for beautiful soup
html = driver.page_source


#create a list of all of the products, and create and empty dictionary
bags = driver.find_elements_by_class_name('product')
product_html_li = []
for bag in bags:
    time.sleep(0.5)
    product_html_li.append(bag.get_attribute('innerHTML'))

product_info_dict = {'product_url':[],
                    'product_name':[],
                    'product_price':[]}

#loop through the lists and append the items
for i in product_html_li:
    if 'marketing-bundle' in i:
        if re.search('.*?href\=\"(.*)\".*', i):
            url = re.search('.*?href\=\"(.*)\".*', i)
            product_info_dict_regex['product_url'].append(url.group(1))
            product_info_dict_regex['product_price'].append(price.group(1))
            product_info_dict_regex['product_name'].append(name.group(1))
        elif re.search('.*?href\=\"(.*)\"\>\<div class\=\"product-grid-xmedia.*', i):
            url = re.search('.*?href\=\"(.*)\"\>\<div class\=\"product-grid-xmedia.*', i)
            product_info_dict_regex['product_url'].append(url.group(1))
            product_info_dict_regex['product_price'].append(price.group(1))
            product_info_dict_regex['product_name'].append(name.group(1))
        else:
            product_info_dict_regex['product_url'].append('NA')
            product_info_dict_regex['product_price'].append('NA')
            product_info_dict_regex['product_name'].append('NA')
            
    else:
        url = re.search('.*?href\=\"(.*)\"\>\<div class\=\"product-grid-xmedia.*', i)
        name = re.search('.*?alt\=\"(.*)\" src\=\".*', i)
        price = re.search('.*?data-price\=\"(.*)\"\>.*', i)
        product_info_dict_regex['price'].append(price.group(1))
        product_info_dict_regex['description'].append(name.group(1))
        product_info_dict_regex['link'].append(url.group(1))

end_time = time.time()
print('done')
print("Total time: "+ str(end_time - start_time) +'sec')


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver


#### Create a DataFrame of these needed items:

We will create a df of the dictionary.

**steps for cleaning:**
1. clean up item description
2. drop **identical** rows
3. **Drop** rows w/ **NA price** values
4. change **prices to floats**
5. **lower case** the descriptions

In [150]:
zara_bag_df = pd.DataFrame(product_info_dict)

In [151]:
# zara_bag_df= 
zara_bag_df = zara_bag_df[['description','link','price']]
zara_bag_df.head()

,description,link,price
0,ROUND RAFFIA CROSSBODY BAG,https://www.zara.com/us/en/round-raffia-crossb...,35.99 USD
1,WOOD CROSSBODY BAG,https://www.zara.com/us/en/wood-crossbody-bag-...,49.90 USD
2,WOOD CROSSBODY BAG,https://www.zara.com/us/en/wood-crossbody-bag-...,49.90 USD
3,WOODEN CLASP CROSSBODY BAG,https://www.zara.com/us/en/wooden-clasp-crossb...,39.90 USD
4,CROCHETED CROSSBODY BAG WITH TORTOISESHELL CLASP,https://www.zara.com/us/en/crocheted-crossbody...,39.90 USD


##### 1. Fix item description by **removing _' - Item available in more colors'_**

In [152]:
zara_bag_df.description = zara_bag_df.description.apply(lambda x: x.replace(' - Item available in more colors',''))

##### 2. Drop Identical Rows:

Zara usese the same url for bags with multiple colors. So, here we will drop any duplicates.

In [155]:
zara_bag_df.drop_duplicates(inplace=True)

##### 3. Drop Rows with NA values:

In [161]:
zara_bag_df = zara_bag_df[zara_bag_df.price != 'NA']

##### 4. Change the prices to floats:

Currently our prices are in the form dd.dd USD and we would like them to just be a float.

In [163]:
zara_bag_df.price = zara_bag_df.price.apply(lambda x: float(x.replace(' USD','')))

/Users/elenasm7/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


##### 5. Lower Case Descriptions

In [166]:
zara_bag_df.description = zara_bag_df.description.apply(lambda x: x.lower())

/Users/elenasm7/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [167]:
zara_bag_df.head()

,description,link,price
0,round raffia crossbody bag,https://www.zara.com/us/en/round-raffia-crossb...,35.99
1,wood crossbody bag,https://www.zara.com/us/en/wood-crossbody-bag-...,49.90
2,wood crossbody bag,https://www.zara.com/us/en/wood-crossbody-bag-...,49.90
3,wooden clasp crossbody bag,https://www.zara.com/us/en/wooden-clasp-crossb...,39.90
4,crocheted crossbody bag with tortoiseshell clasp,https://www.zara.com/us/en/crocheted-crossbody...,39.90


##### Now, that our DF is nice and clean we will save it as a csv file!

In [168]:
zara_bag_df.to_csv('Zara_bags.csv')

#### Get product Image URLS:

We will use the function **get_image_srcs**, which we defined below to grab the correct image SRC.

In [240]:
image_links = []
def get_image_src_driver(link):
    time.sleep(1)
    driver = webdriver.Chrome(ChromeDriverManager().install()) 
    try:
        driver.get(link)
        bag = driver.find_elements_by_class_name('_seoImg')[1].get_attribute('innerHTML')
        time.sleep(1)
        driver.close()
        groups = re.search('.*?src\=\"(.*)\" alt', bag)
        image_links.append('https:'+groups.group(1))
        print('https:'+groups.group(1))
        return 'https:'+groups.group(1)
    except:
        time.sleep(1)
        driver.close()
        image_links.append('no access')
        return 'no access'
    

In [241]:
zara_bag_df['image_url'] = zara_bag_df.link.apply(lambda x: get_image_src_driver(x))


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/6382/304/111/2/w/1920/6382304111_1_1_1.jpg?ts=1551372196337

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/2614/004/105/4/w/1920/2614004105_1_1_1.jpg?ts=1551256549860

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/2614/004/105/2/w/1920/2614004105_1_1_1.jpg?ts=1551257091077

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/2616/004/202/2/w/1920/2616004202_1_1_1.jpg?ts=1551715282406

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedr

https://static.zara.net/photos///2019/V/1/1/p/3656/004/202/2/w/1920/3656004202_1_1_1.jpg?ts=1553530064771

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/8602/004/202/2/w/1920/8602004202_1_1_1.jpg?ts=1554886464280

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1635/004/050/2/w/1920/1635004050_1_1_1.jpg?ts=1551274203961

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/3453/004/092/2/w/1920/3453004092_1_1_1.jpg?ts=1553765035160

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1686/004/202/2/w/1920/1686004202_1_1_1.jpg?ts=

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/2690/004/194/2/w/1920/2690004194_1_1_1.jpg?ts=1552913539207

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1697/004/202/2/w/1920/1697004202_1_1_1.jpg?ts=1552048537480

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1672/004/040/2/w/1920/1672004040_1_1_1.jpg?ts=1551801777772

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1462/004/040/2/w/1920/1462004040_1_1_1.jpg?ts=1551373362430

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedri

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/2304/004/040/2/w/1920/2304004040_1_1_1.jpg?ts=1551359894116

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/6008/304/020/2/w/1920/6008304020_1_1_1.jpg?ts=1551378887176

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/6081/304/040/4/w/1920/6081304040_1_1_1.jpg?ts=1551355720451

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/8040/204/040/2/w/1920/8040204040_2_2_1.jpg?ts=1551356842682

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedri

https://static.zara.net/photos///2019/V/1/1/p/1664/004/202/2/w/1920/1664004202_1_1_1.jpg?ts=1551378451346

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1664/004/202/4/w/1920/1664004202_1_1_1.jpg?ts=1551378155218

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1641/004/105/2/w/1920/1641004105_1_1_1.jpg?ts=1551702039528

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1651/004/032/2/w/1920/1651004032_1_1_1.jpg?ts=1551702983680

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedr

...

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/5018/304/040/2/w/1920/5018304040_1_1_1.jpg?ts=1551354744526

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/2652/004/405/2/w/1920/2652004405_1_1_1.jpg?ts=1555076461991

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/3318/004/087/2/w/1920/3318004087_1_1_1.jpg?ts=1551454248915

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/3319/004/037/2/w/1920/3319004037_1_1_1.jpg?ts=1551886926822

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedri

https://static.zara.net/photos///2019/V/1/1/p/1130/004/131/2/w/1920/1130004131_1_1_1.jpg?ts=1551295714372

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/2086/004/040/2/w/1920/2086004040_1_1_1.jpg?ts=1551359333904

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1412/004/050/2/w/1920/1412004050_1_1_1.jpg?ts=1551300683186

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1410/004/040/2/w/1920/1410004040_1_1_1.jpg?ts=1554910997667

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1324/304/040/2/w/1920/1324304040_1_1_1.jpg?ts=

https://static.zara.net/photos///2019/V/1/1/p/3522/004/050/2/w/1920/3522004050_1_1_1.jpg?ts=1551343558503

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/3521/004/202/2/w/1920/3521004202_1_1_1.jpg?ts=1551342959703

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/3364/004/040/2/w/1920/3364004040_1_1_1.jpg?ts=1551442734627

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/3320/004/202/2/w/1920/3320004202_1_1_1.jpg?ts=1551712710542

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/3322/004/202/2/w/1920/3322004202_1_1_1.jpg?ts=

https://static.zara.net/photos///2019/V/0/1/p/7056/004/303/2/w/1920/7056004303_2_1_1.jpg?ts=1553799925488

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/0/1/p/4751/002/050/2/w/1920/4751002050_2_1_1.jpg?ts=1553613833483

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/0/1/p/6466/002/038/2/w/1920/6466002038_2_1_1.jpg?ts=1548347958204

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/5306/304/102/2/w/1920/5306304102_1_1_1.jpg?ts=1551377342032

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver
https://static.zara.net/photos///2019/V/1/1/p/1621/004/040/2/w/1920/1621004040_1_1_1.jpg?ts=

/Users/elenasm7/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
zara_bag_df.image_url.value_counts()

__Let's drop all of the 'No access' rows. There are only 9, so we won't be missing much.__

In [246]:
zara_bag_df = zara_bag_df[zara_bag_df.image_url != 'no access']
zara_bag_df.head()

,description,link,price,image_url
0,round raffia crossbody bag,https://www.zara.com/us/en/round-raffia-crossb...,35.99,https://static.zara.net/photos///2019/V/1/1/p/...
1,wood crossbody bag,https://www.zara.com/us/en/wood-crossbody-bag-...,49.90,https://static.zara.net/photos///2019/V/1/1/p/...
2,wood crossbody bag,https://www.zara.com/us/en/wood-crossbody-bag-...,49.90,https://static.zara.net/photos///2019/V/1/1/p/...
3,wooden clasp crossbody bag,https://www.zara.com/us/en/wooden-clasp-crossb...,39.90,https://static.zara.net/photos///2019/V/1/1/p/...
4,crocheted crossbody bag with tortoiseshell clasp,https://www.zara.com/us/en/crocheted-crossbody...,39.90,https://static.zara.net/photos///2019/V/1/1/p/...


In [247]:
zara_bag_df.to_csv('zara_img_links_df.csv')

#### Save the Images:

From the URL lists created above we will use the 'save_src_image' function we defined to save the images to our folder.

In [359]:
from PIL import Image
import requests
from io import BytesIO

def save_src_image_apply(url,image_names):
    start_time = time.time()
    time.sleep(1)
    try:
        global c
        c += 1
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        image_save_loc = 'zara_bags/'+'za_'+str(c)+'.jpg'
        img.save(image_save_loc, "JPEG")
        image_names.append("za_"+ str(c)+".png")
        print("Saved "+"za_{}".format(c)+".png")
        print(c)
        return "za_"+ str(c)+".png"
    except Exception as e:
        image_names.append(str(e))
        print(str(e))
        return e

In [360]:
c = 0
image_names_new =[]
zara_bag_df['img'] = zara_bag_df.image_url.apply(lambda x: save_src_image_apply(x,image_names_new))

Saved za_1.png
1
Saved za_2.png
2
Saved za_3.png
3
Saved za_4.png
4
Saved za_5.png
5
Saved za_6.png
6
Saved za_7.png
7
Saved za_8.png
8
Saved za_9.png
9
Saved za_10.png
10
Saved za_11.png
11
Saved za_12.png
12
Saved za_13.png
13
Saved za_14.png
14
Saved za_15.png
15
Saved za_16.png
16
Saved za_17.png
17
Saved za_18.png
18
Saved za_19.png
19
Saved za_20.png
20
Saved za_21.png
21
Saved za_22.png
22
Saved za_23.png
23
Saved za_24.png
24
Saved za_25.png
25
Saved za_26.png
26
Saved za_27.png
27
Saved za_28.png
28
Saved za_29.png
29
Saved za_30.png
30
Saved za_31.png
31
Saved za_32.png
32
Saved za_33.png
33
Saved za_34.png
34
Saved za_35.png
35
Saved za_36.png
36
Saved za_37.png
37
Saved za_38.png
38
Saved za_39.png
39
Saved za_40.png
40
Saved za_41.png
41
Saved za_42.png
42
Saved za_43.png
43
Saved za_44.png
44
Saved za_45.png
45
Saved za_46.png
46
Saved za_47.png
47
Saved za_48.png
48
Saved za_49.png
49
Saved za_50.png
50
Saved za_51.png
51
Saved za_52.png
52
Saved za_53.png
53
Saved za_54

/Users/elenasm7/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [364]:
zara_bag_df.to_csv('zara_df_final.csv')

In [389]:
list(zara_bag_df.img.value_counts().index)

['za_120.png',
 'za_40.png',
 'za_265.png',
 'za_172.png',
 'za_94.png',
 'za_257.png',
 'za_294.png',
 'za_278.png',
 'za_80.png',
 'za_39.png',
 'za_254.png',
 'za_234.png',
 'za_14.png',
 'za_144.png',
 'za_131.png',
 'za_112.png',
 'za_113.png',
 'za_89.png',
 'za_143.png',
 'za_218.png',
 'za_19.png',
 'za_289.png',
 'za_283.png',
 'za_142.png',
 'za_124.png',
 'za_150.png',
 'za_287.png',
 'za_109.png',
 'za_9.png',
 'za_182.png',
 'za_146.png',
 'za_196.png',
 'za_198.png',
 'za_31.png',
 'za_149.png',
 'za_186.png',
 'za_18.png',
 'za_240.png',
 'za_316.png',
 'za_88.png',
 'za_133.png',
 'za_26.png',
 'za_307.png',
 'za_106.png',
 'za_163.png',
 'za_46.png',
 'za_272.png',
 'za_141.png',
 'za_314.png',
 'za_247.png',
 'za_165.png',
 'za_10.png',
 'za_205.png',
 'za_135.png',
 'za_2.png',
 'za_261.png',
 'za_194.png',
 'za_311.png',
 'za_246.png',
 'za_211.png',
 'za_72.png',
 'za_51.png',
 'za_168.png',
 'za_11.png',
 'za_153.png',
 'za_69.png',
 'za_74.png',
 'za_178.png',
 '

In [404]:
zara_bag_df = zara_bag_df[zara_bag_df.img != ('Failed to parse: https:data:image')].drop(['Unnamed: 0'],axis=1)

In [405]:
zara_bag_df.to_csv('zara_df_final_2.csv')

In [409]:
zara_bag_df.reset_index(inplace=True)
zara_bag_df.drop(['index'],axis=1,inplace=True)
zara_bag_df.head()

,level_0,description,link,price,image_url,img
0,0,round raffia crossbody bag,https://www.zara.com/us/en/round-raffia-crossb...,35.99,https://static.zara.net/photos///2019/V/1/1/p/...,za_1.png
1,1,wood crossbody bag,https://www.zara.com/us/en/wood-crossbody-bag-...,49.90,https://static.zara.net/photos///2019/V/1/1/p/...,za_2.png
2,2,wood crossbody bag,https://www.zara.com/us/en/wood-crossbody-bag-...,49.90,https://static.zara.net/photos///2019/V/1/1/p/...,za_3.png
3,3,wooden clasp crossbody bag,https://www.zara.com/us/en/wooden-clasp-crossb...,39.90,https://static.zara.net/photos///2019/V/1/1/p/...,za_4.png
4,4,crocheted crossbody bag with tortoiseshell clasp,https://www.zara.com/us/en/crocheted-crossbody...,39.90,https://static.zara.net/photos///2019/V/1/1/p/...,za_5.png


In [ ]:
zara_bag_df['company'] = 'Zara'

In [414]:
zara_bag_df.to_csv('zara_df_final_2.csv')